# Libraries/Imports

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!git config --global user.email "chau.nguyenminh109@gmail.com"
!git config --global user.name "chloe-nguyenminh"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install datasets evaluate transformers[sentencepiece]
%pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
%pip install kagglehub
%pip install transformers
%pip install accelerate
%pip install --upgrade torch torchvision torchaudio
%pip install --upgrade torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
ERROR: torch_xla-1.9-cp37-cp37m-linux_x86_64.whl is not a supported wheel on this 

In [5]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import torch

In [6]:
def print_rows(dataset, start=0, end=10, split="", feature=""):
  if split == "":
    for split in dataset:
      print(f"Entries {start+1} - {end} of the {split} data:")
      for i in range(start, end):
        if feature:  # If feature is provided
          print(f"{feature}: {dataset[split][i][feature]}")  # Print the specified feature
        else:
          print(dataset[split][i])  # Print the entire row if no feature is specified
      print("-" * 20)
  else:
    print(f"Entries {start+1} - {end} of the {split} data:")
    for i in range(start, end):
      if feature:  # If feature is provided
        print(f"{feature}: {dataset[split][i][feature]}")  # Print the specified feature
      else:
        print(dataset[split][i])  # Print the entire row if no feature is specified
    print("-" * 20)

# Preprocessing

In [7]:
# Preprocess COT dataset
from datasets import load_dataset

cot_ds = load_dataset("AI-MO/NuminaMath-CoT")

cot_ds['train'] = cot_ds['train'].remove_columns(['messages'])
cot_ds['test'] = cot_ds['test'].remove_columns(['messages'])
cot_ds['train'] = cot_ds['train'].remove_columns(['source'])
cot_ds['test'] = cot_ds['test'].remove_columns(['source'])

# Remove chinese characters from COT dataset
import re

def contains_chinese(text):
    # match Chinese characters
    pattern = re.compile(r'[\u4e00-\u9fff\u2e80-\u2eff\u31c0-\u31ef\uff00-\uffef]')
    return bool(pattern.search(text))

def filter_entries(dataset, fields):
    # Filter out entries that contain Chinese characters
    filtered_dataset = dataset.filter(lambda example: not any(contains_chinese(example[field]) for field in fields))
    return filtered_dataset

# remove entries with Chinese characters
fields_to_check = ['problem', 'solution']
cot_ds['train'] = filter_entries(cot_ds['train'], fields_to_check)
cot_ds['test'] = filter_entries(cot_ds['test'], fields_to_check)
print(cot_ds)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/859494 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/859494 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution'],
        num_rows: 850151
    })
    test: Dataset({
        features: ['problem', 'solution'],
        num_rows: 100
    })
})


In [8]:
# Preprocess MATH dataset (load all json files into into Dataset object)

import kagglehub

# Download latest version
path = kagglehub.dataset_download("mathurinache/math-dataset")

print("Path to dataset files:", path)

import os
from datasets import Dataset, DatasetDict

def load_json_files(data_dir):
    """Loads JSON files from a directory into a Dataset."""
    all_data = []
    problems = 0
    for subdir in os.listdir(data_dir):
      subdir_path = os.path.join(data_dir, subdir)
      for filename in os.listdir(subdir_path):
        if filename.endswith(".json"):
          problems += 1
          filepath = os.path.join(subdir_path, filename)
          with open(filepath, "r") as f:
            all_data.append(json.load(f))
    # Create a Pandas DataFrame to easily convert into a Dataset\
    print(f"Loaded {problems} problems.")
    return all_data

# Assuming 'path' is from kagglehub.dataset_download
math_dir = os.path.join(path, "MATH")
train_dir = os.path.join(math_dir, "train")
test_dir = os.path.join(math_dir, "test")

train_data = load_json_files(train_dir)
test_data = load_json_files(test_dir)

# Convert the train and test data into Dataset objects
train_dataset = Dataset.from_dict({
    "problem": [item["problem"] for item in train_data],
    # "level": [item["level"] for item in train_data],
    # "type": [item["type"] for item in train_data],
    "solution": [item["solution"] for item in train_data]
})

test_dataset = Dataset.from_dict({
    "problem": [item["problem"] for item in test_data],
    # "level": [item["level"] for item in test_data],
    # "type": [item["type"] for item in test_data],
    "solution": [item["solution"] for item in test_data]
})

math_ds = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(math_ds)


100%|██████████| 7.07M/7.07M [00:01<00:00, 5.20MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mathurinache/math-dataset/versions/1
Loaded 7500 problems.
Loaded 5000 problems.
DatasetDict({
    train: Dataset({
        features: ['problem', 'solution'],
        num_rows: 7500
    })
    test: Dataset({
        features: ['problem', 'solution'],
        num_rows: 5000
    })
})


In [9]:
# Concatenate and split datasets
from datasets import concatenate_datasets

# Make validation dataset
train_valid_split = cot_ds['train'].train_test_split(test_size=0.1)
cot_ds['train'] = train_valid_split['train']
cot_ds['test'] = train_valid_split['test']

train_valid_split = cot_ds['train'].train_test_split(test_size=0.12)
cot_ds['train'] = train_valid_split['train']
cot_ds['validation'] = train_valid_split['test']

# Add MATH dataset as test dataset
merged_math = concatenate_datasets([math_ds['train'], math_ds['test']])
cot_ds['test'] = concatenate_datasets([cot_ds['test'], merged_math])

ds = cot_ds
print(ds)

del cot_ds
del math_ds

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution'],
        num_rows: 673318
    })
    test: Dataset({
        features: ['problem', 'solution'],
        num_rows: 97516
    })
    validation: Dataset({
        features: ['problem', 'solution'],
        num_rows: 91817
    })
})


In [10]:
print("Split")
print("train:", len(ds['train']) / ( len(ds['train']) + len(ds['validation']) + len(ds['test']) ))
print("test:", len(ds['test']) / ( len(ds['train']) + len(ds['validation']) + len(ds['test']) ))
print("validation:", len(ds['validation']) / ( len(ds['train']) + len(ds['validation']) + len(ds['test']) ))

Split
train: 0.7805219028320839
test: 0.11304223840232029
validation: 0.10643585876559582


# Embedding

In [12]:
ds32 = DatasetDict({
    split: dataset.select(range(10000))
    for split, dataset in ds.items()
})
print(ds32)

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['problem', 'solution'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['problem', 'solution'],
        num_rows: 10000
    })
})


In [13]:
# Generate Embeddings

from transformers import AutoTokenizer, AutoModel
import torch

# Load MathBERT
model_name = "tbs17/MathBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True)
model = AutoModel.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/441M [00:00<?, ?B/s]

In [14]:
sample_sentence = 'Let \\( T = 1 - 2 + 3 - 4 + \\cdots + 2022 - 2023 \\). What is the residue of \\( T \\), modulo 2024?'
sample_sentence_ids = tokenizer(sample_sentence, add_special_tokens=True,
        truncation=True,
        max_length=128,).input_ids
print(sample_sentence_ids)


[101, 2292, 1032, 1006, 1056, 1027, 1015, 1011, 1016, 1009, 1017, 1011, 1018, 1009, 1032, 3729, 12868, 1009, 16798, 2475, 1011, 16798, 2509, 1032, 1007, 1012, 2054, 2003, 1996, 21755, 1997, 1032, 1006, 1056, 1032, 1007, 1010, 16913, 18845, 16798, 2549, 1029, 102]


In [15]:
# Function to compute embeddings
def compute_embeddings(batch):
    # Tokenize the problem text
    inputs = tokenizer(
        batch["problem"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt")

    # Move input tensors to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token embedding (typically the first token)
    cls_embeddings = outputs.last_hidden_state.squeeze(0).cpu().numpy()

    # Return embeddings
    return {"embeddings": cls_embeddings}

# Apply the function to each split of the dataset
def process_dataset(dataset_dict):
    for split in dataset_dict:
        dataset_dict[split] = dataset_dict[split].map(compute_embeddings, batched=True, batch_size=32)
        dataset_dict[split].set_format(type="torch", columns=["embeddings"], output_all_columns=True)
    return dataset_dict


# Add embeddings to the dataset
ds32 = process_dataset(ds32)

# Save embeddings or access them
torch.save(model.state_dict(), 'mathbert_weights.pth')
# model.load_state_dict(torch.load('mathbert_weights.pth'))

print(ds32)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution', 'embeddings'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['problem', 'solution', 'embeddings'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['problem', 'solution', 'embeddings'],
        num_rows: 10000
    })
})


In [59]:
vocab_size = len(tokenizer)
print(vocab_size)

30522


In [16]:
batch_embeddings = []
for i in range(len(ds32['train'])):
  p= torch.tensor(ds32['train'][i]["embeddings"]).unsqueeze(0)
  batch_embeddings.append(p)

print(len(batch_embeddings))
batch_embeddings = torch.cat(batch_embeddings, dim=0)
print(batch_embeddings.shape)


<ipython-input-16-85ee846d44e0>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  p= torch.tensor(ds32['train'][i]["embeddings"]).unsqueeze(0)


10000
torch.Size([10000, 128, 768])


# Encoder


In [61]:
print(ds32['train'][0])

{'embeddings': tensor([[-1.1911, -0.9803,  2.1448,  ..., -2.4151, -0.0282,  0.8819],
        [ 1.0272, -1.3826, -0.6176,  ..., -1.7151,  1.1262,  0.4477],
        [ 0.2474,  0.2179,  0.8676,  ..., -0.7816,  0.8544, -0.7041],
        ...,
        [-1.3861, -1.4666,  0.8211,  ..., -1.3715,  0.9256,  1.4897],
        [-0.9546, -0.9264,  0.2426,  ..., -1.3892,  0.6853,  1.5767],
        [-0.4706, -1.9400,  0.8370,  ..., -1.4447,  1.0010,  2.5040]]), 'problem': 'April has five different basil plants and three different tomato plants. In how many ways can she arrange the plants in a circular formation if she puts all the tomato plants next to each other?', 'solution': '1. Group the three tomato plants as a single entity. The arrangement then involves the five basil plants and this one group of three tomato plants, making it effectively six entities to arrange.\n2. Since the arrangement is circular, fix one plant and arrange the rest. There are $(6-1)! = 5! = 120$ ways to arrange these groups

In [62]:
train_embeddings = ds32["train"]["embeddings"]

In [63]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from typing import Union, Tuple


In [64]:
from transformers import T5Config
class CustomConfig(T5Config):
    def __init__(
        self,
        d_model=768,      # Hidden size 768   16384
        num_heads=8,       # Number of attention heads
        d_kv = 96,
        d_ff= 2048,         # Feedforward network size
        num_encoder_layers=6,  # Number of encoder layers
        num_decoder_layers=6,  # Number of decoder layers
        dropout_rate=0.1,      # Dropout rate
        attention_dropout_rate=0.1,  # Attention dropout rate
        layer_norm_eps=1e-6,   # Layer norm epsilon for stability
        vocab_size=30522,     # Vocabulary size
        bos_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.pad_token_id,
        max_length=128,

    ):
        super().__init__(vocab_size=vocab_size,
                        d_model=d_model,
                        d_kv=d_kv,
                        d_ff=d_ff,
                        num_heads=num_heads,
                        num_encoder_layers=num_encoder_layers,
                        dropout_rate=dropout_rate,
                        layer_norm_eps=layer_norm_eps,
                        bos_token_id=bos_token_id,
                        eos_token_id=eos_token_id,
                        max_length=max_length
                  )
        self.num_encoder_layers = num_encoder_layers
        self.attention_dropout_rate = attention_dropout_rate
        self.to_json_string = True
        self.decoder_start_token_id = tokenizer.pad_token_id


        assert d_model % num_heads == 0, "d_model mfust be divisible by num_heads"


In [66]:
train_embedding = torch.tensor(ds32['train'][1]["embeddings"])
train_embedding = train_embedding.unsqueeze(0)

print(train_embedding)
print(train_embedding.shape)

tensor([[[-2.0035, -0.5703,  0.7537,  ..., -1.0333,  0.2734,  0.4358],
         [ 1.1261, -0.3969,  0.9340,  ..., -0.9725, -0.5348,  0.1086],
         [ 0.3291,  0.1631,  0.2517,  ..., -0.9325, -0.9904, -0.5742],
         ...,
         [-0.3364, -1.4423,  1.2558,  ..., -1.7150,  0.1065, -0.5303],
         [-0.2104, -1.3901,  0.3626,  ..., -0.1736,  1.2462,  0.0107],
         [-0.0137, -1.8235,  1.2004,  ..., -0.0559,  0.9874,  1.3253]]])
torch.Size([1, 128, 768])


<ipython-input-66-fea8994eb4b5>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_embedding = torch.tensor(ds32['train'][1]["embeddings"])


In [67]:
class FFNLayer(nn.Module):
    # T5LayerFF
    def __init__(self, config):
        super().__init__()
        self.linear1 = nn.Linear(config.d_model, config.d_ff)  # First projection
        self.activation = nn.GELU()  # Non-linear activation
        self.linear2 = nn.Linear(config.d_ff, config.d_model)  # Second projection
        self.dropout = nn.Dropout(config.dropout_rate)  # Dropout regularization
        self.layer_norm = nn.LayerNorm(config.d_model, eps=config.layer_norm_eps)  # Layer normalization

    def forward(self, hidden_states):

        #initializing weights in the feed-forward layers to improve stability during training.
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)

         # Input goes through the FFN
        forwarded_states = self.linear1(hidden_states)
        forwarded_states = self.activation(forwarded_states)
        forwarded_states = self.dropout(forwarded_states)
        forwarded_states = self.linear2(forwarded_states)

        # Add residual connection and layer norm
        hidden_states = self.layer_norm(hidden_states + forwarded_states)
        return hidden_states


In [68]:
class AttentionLayer(nn.Module):
    # T5Attention
    def __init__(self, config):
        super().__init__()
        self.num_heads = config.num_heads  # Number of attention heads
        self.d_model = config.d_model
        self.dropout = config.attention_dropout_rate

        self.self_attention = nn.MultiheadAttention(
            embed_dim=self.d_model,
            num_heads=self.num_heads,
            dropout=self.dropout
        )

        # skipping Relative positional bias

        self.layer_norm = nn.LayerNorm(self.d_model, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.dropout_rate)

    def forward(self, hidden_states, attention_mask=None, layer_head_mask=None, output_attentions=False):
        """
        Forward pass for the attention layer.

        Args:
            hidden_states (torch.FloatTensor): Input tensor of shape [seq_len, batch_size, d_model].
            attention_mask (torch.FloatTensor, optional): Mask for attention mechanism of shape [batch_size, seq_len].
            layer_head_mask (torch.FloatTensor, optional): Mask for specific attention heads.
            output_attentions (bool, optional): Whether to return attention scores.

        Returns:
            torch.FloatTensor: Updated hidden states after attention.
            Optional[torch.FloatTensor]: Attention weights if `output_attentions=True`.
        """

        # Transpose hidden_states to [seq_len, batch_size, d_model] for nn.MultiheadAttention
        hidden_states = hidden_states.transpose(0, 1)  # [seq_len, batch_size, d_model]

        # Apply multi-head self-attention
        attention_output, attention_weights = self.self_attention(
            query=hidden_states,
            key=hidden_states,
            value=hidden_states,
            attn_mask=attention_mask
        )


        # Residual connection and layer normalization
        hidden_states = self.layer_norm(hidden_states + self.dropout(attention_output))

        # Transpose back to [batch_size, seq_len, d_model]
        hidden_states = hidden_states.transpose(0, 1)


        if output_attentions:
            return hidden_states, attention_weights
        return hidden_states

In [69]:
class EncoderBlock(nn.Module):
   # a single Transformer encoder layer
   # similar to T5 Block
    def __init__(self, config):
        super(EncoderBlock, self).__init__()
        self.layer = nn.ModuleList()
        self.layer.append(AttentionLayer(config))
        self.layer.append(FFNLayer(config))


    def forward(
        self,
        hidden_states,
        attention_mask=None,
        # position_bias=None,
        layer_head_mask=None,
        #cross_attn_layer_head_mask=None,
        past_key_value=None,
        use_cache=False,
        output_attentions=False,
        return_dict=True,
        cache_position=None,
    ):
        """
        Forward pass for the encoder block.
        Args:
            hidden_states: Tensor of shape [?].
            attention_mask: Mask to prevent attention to certain positions.
            layer_head_mask: Mask for specific attention heads.
            use_cache: Whether to use caching for inference.
            output_attentions: Whether to return attention outputs.
            cache_position: Cache tracking the current position for inference.
        Returns:
            hidden_states: Updated hidden states after attention and FFN.
            outputs: A tuple containing additional optional outputs.
        """

        self_attention_outputs = self.layer[0](
            hidden_states,
            attention_mask=attention_mask,
           # layer_head_mask=layer_head_mask,
            #use_cache=use_cache,
            #output_attentions=output_attentions,
           # cache_position=cache_position,
        )
        if output_attentions:
            attention_outputs = self_attention_outputs[1:]
            hidden_states = self_attention_outputs[0]
        else:
            hidden_states = self_attention_outputs

        hidden_states = self.layer[1](hidden_states)

        # Clamp to handle FP16 training again
        if hidden_states.dtype == torch.float16:
            clamp_value = torch.finfo(hidden_states.dtype).max - 1000
            hidden_states = torch.clamp(hidden_states, min=-clamp_value, max=clamp_value)

        # Prepare outputs
        #outputs = (hidden_states,) + attention_outputs

        return hidden_states

# Testing
# Initialize EncoderBlock
config = CustomConfig(d_model=768, num_heads=8, d_ff=2048)
encoder_block = EncoderBlock(config)

# Pass through EncoderBlock
block_output = encoder_block(hidden_states=train_embedding)
hidden_states= block_output  # Extract the hidden states
print(f"Output Shape from EncoderBlock: {hidden_states.shape}")
print(f"Output from EncoderBlock: {hidden_states}")


Output Shape from EncoderBlock: torch.Size([1, 128, 768])
Output from EncoderBlock: tensor([[[-2.3037,  0.1918,  0.7524,  ..., -0.8393, -0.1329,  0.7979],
         [ 0.1780, -0.2506,  0.3723,  ..., -0.7957,  0.2541, -0.4301],
         [ 0.0330,  0.2383,  0.6142,  ..., -0.4496, -1.6704, -0.9315],
         ...,
         [-1.0288, -1.4762,  0.5263,  ..., -1.0063,  0.0914, -0.7879],
         [-0.0541, -1.4516,  0.2612,  ...,  0.2945,  1.0083,  0.3182],
         [-0.2334, -1.2355,  1.6593,  ...,  0.1557,  1.1018, -0.1131]]],
       grad_fn=<NativeLayerNormBackward0>)


In [70]:
class TransformerEncoder(nn.Module):
  # stacks multiple encoder layers
    def __init__(self, config):
        super(TransformerEncoder, self).__init__()

        self.block = nn.ModuleList(
            [EncoderBlock(config) for i in range(config.num_encoder_layers)]
        )
        self.final_layer_norm = nn.LayerNorm(config.d_model, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.dropout_rate)


    def forward(
          self,
          hidden_states,
          attention_mask=None,
          head_mask=None,
          output_attentions=False,
          output_hidden_states=False,
          return_dict=True,
          ):
          """
          Forward pass for the TransformerEncoder.

          Args:
              hidden_states (Tensor): Input embeddings of shape [batch_size, seq_len, d_model].
              attention_mask (Tensor, optional): Mask to prevent attention to certain positions (e.g., padding).
              head_mask (Tensor, optional): Mask for specific attention heads.
              output_attentions (bool, optional): Whether to return attention weights.
              output_hidden_states (bool, optional): Whether to return intermediate hidden states.
              return_dict (bool, optional): Whether to return outputs as a dictionary.

          Returns:
              dict or tuple: Updated hidden states and optional outputs.
          """

          # Prepare outputs if needed
          all_hidden_states = () if output_hidden_states else None
          all_attentions = () if output_attentions else None

          # Apply dropout to input hidden states
          hidden_states = self.dropout(hidden_states)

          # Pass through each layer in the encoder block
          for i, layer_module in enumerate(self.block):
              if output_hidden_states:
                  all_hidden_states = all_hidden_states + (hidden_states,)

              # Forward pass through the encoder block
              # print(hidden_states.shape)
              layer_outputs = layer_module(
                  hidden_states,
                  attention_mask=attention_mask,
                  layer_head_mask=head_mask[i] if head_mask is not None else None,
                  output_attentions=output_attentions,
              )

              # Collect attention weights if requested
              if output_attentions:
                  hidden_states = layer_outputs[0]
                  all_attentions = all_attentions + (layer_outputs[1],)
              else:
                  hidden_states = layer_outputs

          # Apply final layer normalization
          hidden_states = self.final_layer_norm(hidden_states)

          # Apply final dropout
          hidden_states = self.dropout(hidden_states)

          # Add the final hidden state to outputs if requested
          if output_hidden_states:
              all_hidden_states = all_hidden_states + (hidden_states,)

          # Prepare return values
          if not return_dict:
              return tuple(
                  v for v in [hidden_states, all_hidden_states, all_attentions] if v is not None
              )

          return {
              "last_hidden_state": hidden_states,
              "hidden_states": all_hidden_states,
              "attentions": all_attentions,
          }

# Testing
# # Initialize TransformerEncoder
# config = CustomConfig(d_model=768, num_heads=8, d_ff=2048, num_encoder_layers=6)
# transformer_encoder = TransformerEncoder(config)
# attention_mask = torch.ones(batch_embeddings.shape[1], batch_embeddings.shape[1])

# # Pass through TransformerEncoder
# encoder_output = transformer_encoder(hidden_states=train_embedding, attention_mask=attention_mask)

# # Print the last hidden state
# print(f"Output Shape from TransformerEncoder: {encoder_output['last_hidden_state'].shape}")
# print(f"Last Hidden State: {encoder_output['last_hidden_state']}")



In [71]:
class EncoderModel(nn.Module):
  # wraps the transformer encoder
  # similar to T5EncoderModel


    def __init__(self, config):
      super(EncoderModel, self).__init__()
      # Embedding layer (token + positional)
      # self.token_embedding = nn.Embedding(config., hidden_size)
      # self.position_embedding = nn.Embedding(max_seq_len, hidden_size)
      # self.dropout = nn.Dropout(dropout)
      self.encoder = TransformerEncoder(config)
      # self.hidden_size = config.h


    def forward(
        self,
        hidden_states: torch.FloatTensor,  # Precomputed embeddings
        attention_mask = None,
        output_attentions = False,
        output_hidden_states = False,
        return_dict = True,
    ) -> Union[Tuple[torch.FloatTensor], dict]:
      """
      Forward pass for the EncoderModel.

      Args:
          hidden_states (torch.FloatTensor): Precomputed embeddings of shape [batch_size, seq_len, d_model].
          attention_mask (Optional[torch.FloatTensor]): Mask of shape [batch_size, seq_len] to prevent attention to padding tokens.
          output_attentions (Optional[bool]): Whether to return attention weights.
          output_hidden_states (Optional[bool]): Whether to return hidden states of all layers.
          return_dict (Optional[bool]): Whether to return outputs as a dictionary.

      Returns:
          Union[Tuple, dict]: Final hidden states and optional outputs (attention weights, hidden states).
      """

      # Forward pass through the transformer encoder
      # print(hidden_states.shape) # torch.Size([batch_size, 128, 768])
      encoder_outputs = self.encoder(
          hidden_states=hidden_states,
          attention_mask=attention_mask,
          output_attentions=output_attentions,
          output_hidden_states=output_hidden_states,
          return_dict=return_dict,
      )

      return encoder_outputs


In [72]:
# Pass the embedding through the encoder
config = CustomConfig()
encoder_model = EncoderModel(config)
output = encoder_model(train_embeddings)
print(output)

{'last_hidden_state': tensor([[[-1.5275e+00,  1.7398e-02, -5.0637e-01,  ..., -3.2331e-01,
          -3.3861e-01,  4.2824e-01],
         [-7.5746e-01,  1.1584e+00, -4.8278e-01,  ..., -1.2363e+00,
           9.5529e-01,  3.4992e-01],
         [-1.8375e+00,  1.9445e+00,  7.0584e-01,  ...,  8.4859e-02,
           7.3289e-01,  1.3132e+00],
         ...,
         [-0.0000e+00,  0.0000e+00, -3.6957e-01,  ..., -3.3989e-01,
           0.0000e+00,  8.9641e-01],
         [-1.6073e+00,  0.0000e+00, -1.6479e+00,  ...,  3.4307e-01,
           1.2240e+00,  1.7058e+00],
         [-1.4139e+00,  3.5583e-01, -2.1397e-01,  ..., -0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-1.6971e+00, -5.7095e-01, -3.4971e-01,  ...,  9.2488e-01,
          -9.0541e-02, -1.3090e+00],
         [-2.2500e+00, -2.7591e-01,  1.0242e+00,  ..., -1.1881e+00,
          -3.7417e-01,  2.8822e-01],
         [-0.0000e+00, -1.6719e-01,  5.1011e-01,  ..., -5.8991e-01,
          -7.0706e-01, -1.5465e+00],
         ...,
   

In [73]:
encoder_last_hidden_states = output['last_hidden_state']
print(encoder_last_hidden_states.shape) # Expect [batch_size, max_length, 768]

torch.Size([32, 128, 768])


# Decoder



In [74]:
class CrossAttentionLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_heads = config.num_heads
        self.d_model = config.d_model
        self.dropout = config.attention_dropout_rate

        self.cross_attention = nn.MultiheadAttention(
            embed_dim=self.d_model,
            num_heads=self.num_heads,
            dropout=self.dropout
        )

        self.layer_norm = nn.LayerNorm(self.d_model, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.dropout_rate)

    def forward(self, hidden_states, encoder_hidden_states, attention_mask=None):
        # Transpose from [batch_size, seq_len, hidden_dim] to [seq_len, batch_size, hidden_dim]
        hidden_states = hidden_states.transpose(0, 1)
        encoder_hidden_states = encoder_hidden_states.transpose(0, 1)

        # Adjust attention mask if provided
        if attention_mask is not None:
            # Convert attention mask to the correct shape and dtype
            attention_mask = attention_mask.to(dtype=hidden_states.dtype)

            # Ensure the mask has the correct shape (tgt_len, src_len)
            if attention_mask.dim() == 2:
                attention_mask = attention_mask.transpose(0, 1)

            # Convert mask to additive attention mask
            attention_mask = attention_mask.masked_fill(attention_mask == 0, float('-inf'))
            attention_mask = attention_mask.masked_fill(attention_mask == 1, float(0.0))

        attention_output, attention_weights = self.cross_attention(
            query=hidden_states,
            key=encoder_hidden_states,
            value=encoder_hidden_states,
            attn_mask=attention_mask
        )

        hidden_states = self.layer_norm(hidden_states + self.dropout(attention_output))
        # Transpose back to [batch_size, seq_len, hidden_dim]
        hidden_states = hidden_states.transpose(0, 1)

        return hidden_states

class DecoderBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.self_attention = AttentionLayer(config)
        self.cross_attention = CrossAttentionLayer(config)
        self.ffn = FFNLayer(config)

    def forward(
        self,
        hidden_states,
        encoder_hidden_states,
        attention_mask=None,
        encoder_attention_mask=None,
        layer_head_mask=None,
        cross_attn_layer_head_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        # Self Attention
        self_attention_outputs = self.self_attention(
            hidden_states,
            attention_mask=attention_mask,
            layer_head_mask=layer_head_mask,
            output_attentions=output_attentions
        )
        hidden_states = self_attention_outputs if not output_attentions else self_attention_outputs[0]

        # Cross Attention
        cross_attention_outputs = self.cross_attention(
            hidden_states,
            encoder_hidden_states,
            attention_mask=encoder_attention_mask
        )
        hidden_states = cross_attention_outputs

        # Feed Forward
        hidden_states = self.ffn(hidden_states)

        return hidden_states

In [75]:
class TransformerDecoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.block = nn.ModuleList(
            [DecoderBlock(config) for _ in range(config.num_decoder_layers)]
        )
        self.final_layer_norm = nn.LayerNorm(config.d_model, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.dropout_rate)
        self.output_projection = nn.Linear(config.d_model, config.vocab_size, bias=False)

    def forward(
        self,
        hidden_states,
        encoder_hidden_states,
        attention_mask=None,
        encoder_attention_mask=None,
        head_mask=None,
        cross_attn_head_mask=None,
        past_key_values=None,
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True,
    ):
        all_hidden_states = () if output_hidden_states else None
        all_self_attns = () if output_attentions else None
        all_cross_attns = () if output_attentions else None

        hidden_states = self.dropout(hidden_states)

        for i, layer_module in enumerate(self.block):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            cross_attn_layer_head_mask = (
                cross_attn_head_mask[i] if cross_attn_head_mask is not None else None
            )

            hidden_states = layer_module(
                hidden_states,
                encoder_hidden_states,
                attention_mask=attention_mask,
                encoder_attention_mask=encoder_attention_mask,
                layer_head_mask=layer_head_mask,
                cross_attn_layer_head_mask=cross_attn_layer_head_mask,
                output_attentions=output_attentions,
            )

        # Final layer norm
        hidden_states = self.final_layer_norm(hidden_states)
        hidden_states = self.dropout(hidden_states)

        # Project to vocabulary
        logits = self.output_projection(hidden_states)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(v for v in [
                logits,
                all_hidden_states,
                all_self_attns,
                all_cross_attns
            ] if v is not None)

        return {
            "logits": logits,
            "hidden_states": all_hidden_states,
            "self_attentions": all_self_attns,
            "cross_attentions": all_cross_attns,
        }


In [78]:
config = CustomConfig(d_model=768, num_heads=8, d_ff=2048,
                         num_encoder_layers=6, num_decoder_layers=6)

    # Initialize decoder
decoder = TransformerDecoder(config)

# Create sample inputs
decoder_input = batch_embeddings
encoder_hidden_states = batch_embeddings  # Simulating encoder output

# Create attention masks
attention_mask = torch.ones(batch_embeddings.size(1), batch_embeddings.size(1))
encoder_attention_mask = torch.ones(batch_embeddings.size(1), batch_embeddings.size(1))

# Forward pass
decoder_output = decoder(
    hidden_states=train_embeddings,
    encoder_hidden_states=encoder_last_hidden_states,
    attention_mask=attention_mask,
    encoder_attention_mask=encoder_attention_mask,
)


# Model

In [77]:
from transformers.modeling_outputs import Seq2SeqQuestionAnsweringModelOutput
from torch.nn import CrossEntropyLoss
class T5Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.decoder = TransformerDecoder(config)
        self.config = config

    def loss_func(self, inputs, logits):
        # Shift so that tokens < n predict n
        shift_labels = inputs[..., 1:].contiguous()
        shift_logits = logits[..., :-1, :].contiguous()
        # Calculate per-token loss
        loss_fct = CrossEntropyLoss(reduce=False)
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        # Resize and average loss per sample
        loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
        return loss_per_sample.mean()

    def forward(
        self,
        encoder_input,
        decoder_input,
        attention_mask=None,
        decoder_attention_mask=None,
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True,
    ):
        # Encode
        encoder_outputs = self.encoder(
            hidden_states=encoder_input,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # Decode
        decoder_outputs = self.decoder(
            hidden_states=decoder_input,
            encoder_hidden_states=encoder_outputs["last_hidden_state"],
            attention_mask=decoder_attention_mask,
            encoder_attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        if not return_dict:
            return (decoder_outputs["logits"],)

        logits = decoder_outputs['logits']
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        loss = self.loss_func(decoder_input, logits)
        outputs = Seq2SeqQuestionAnsweringModelOutput(
            start_logits=start_logits,
            end_logits=end_logits,
        )

        return outputs


In [80]:
model = T5Model(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"T5 size: {model_size/1000**2:.1f}M parameters")

T5 size: 109.5M parameters


# **Set Up Training**

In [82]:
# Evaluation Metrics

def get_grouped_params(model, no_decay=['bias', 'LayerNorm.weight'], weight_decay=0.01):
  params_with_wd, params_without_wd = [], []
  for n, p in model.named_parameters():
      if any(nd in n for nd in no_decay):
          params_without_wd.append(p)
      else:
          params_with_wd.append(p)
  return [
      {"params": params_with_wd, "weight_decay": weight_decay},
      {"params": params_without_wd, "weight_decay": 0.0},
  ]

In [83]:
# DataLoader
from torch.utils.data.dataloader import DataLoader

ds32.set_format('torch')
train_dataloader = DataLoader(ds32["train"], batch_size=32, shuffle=True)
val_dataloader = DataLoader(ds32["validation"], batch_size=32)

In [84]:
# Optimizer
from torch.optim import AdamW

optimizer = AdamW(get_grouped_params(t5_model), lr=5e-4)

In [21]:
class Inference(Dataset):

    def __init__(self, dataframe, tokenizer, source_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.soln = self.data.soln

    def __len__(self):
        return len(self.soln)

    def __getitem__(self, index):
        soln = str(self.soln[index])
        soln = ' '.join(soln.split())

        source = self.tokenizer.batch_encode_plus([soln], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt',truncation=True)

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
        }

In [17]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    epoch_loss = 0
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(loader)

In [18]:
def evaluate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    epoch_loss = 0
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            yt_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            preds = model(input_ids = ids,attention_mask = mask,decoder_input_ids=yt_ids, lm_labels=lm_labels)
            val_loss = preds[0]
            epoch_loss += val_loss.item()

    return epoch_loss / len(loader)

In [19]:
def infer(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            if _%100==0:
                print(f'Completed {_}')
    return preds

In [20]:
import random
import math
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
TRAIN_EPOCHS = 2
VAL_EPOCHS = 1
LEARNING_RATE = 1e-4
SEED = 2024
MAX_LEN = 128

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed_all(SEED)

best_valid_score = float('-inf')

for epoch in range(TRAIN_EPOCHS):
    train_loss= train(epoch, tokenizer, model, device, train_dataloader, optimizer)
    valid_loss = evaluate(epoch, tokenizer, model, device, val_dataloader)
    torch.save(model.state_dict(), 't5.pt_{}'.format(epoch))
    print(f'\tTrain Loss: {train_loss:.3f} | perplexity: {math.exp(train_loss):7.3f}')
    print(f'\tVal Loss: {valid_loss:.3f}' )


Epoch: 01 | Time: 34m 17s
	Train Loss: 3.458 | Train PPL:  31.683
	Val Loss: 2.761

Epoch: 02 | Time: 58m 39s
	Train Loss: 2.879 | Train PPL:  16.496
	Val Loss: 2.357

Epoch: 03 | Time: 83m 1s
	Train Loss: 2.424 | Train PPL:  11.274
	Val Loss: 2.042

Epoch: 04 | Time: 117m 23s
	Train Loss: 2.18 | Train PPL:  8.751
	Val Loss: 1.754

Epoch: 05 | Time: 141m 44s
	Train Loss: 2.023 | Train PPL:  6.847
	Val Loss: 1.514

Epoch: 06 | Time: 176m 6s
	Train Loss: 1.764 | Train PPL:  6.271
	Val Loss: 1.363

Epoch: 07 | Time: 220m 28s
	Train Loss: 1.54 | Train PPL:  5.384
	Val Loss: 1.096

Epoch: 08 | Time: 254m 49s
	Train Loss: 1.465 | Train PPL:  4.9
	Val Loss: 0.912

Epoch: 09 | Time: 289m 11s
	Train Loss: 1.25 | Train PPL:  3.673
	Val Loss: 0.774

Epoch: 10 | Time: 313m 33s
	Train Loss: 1.151 | Train PPL:  3.281
	Val Loss: 0.686

